In [1]:
cd ~/repos/privacy-code-tools

/Users/stein/repos/privacy-code-tools


In [2]:
import os
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import django;django.setup()

from coder.api.models import *

In [3]:
from tqdm.notebook import tqdm
import json

In [15]:
project = Project.objects.all().filter(id=4)[0]
project.id, project.name

(4, '[2022 Data Breach Project] - State Data Security laws')

In [16]:
policies = Policy.objects.all().filter(project=project.id)
states = {p.company_name: p for p in policies}
#states["Delaware"]
codes = {
    p.company_name: 
        PolicyInstance.objects.all().filter(project=project.id, policy_id=p.id)[0]
    for p in policies
}

In [17]:
codings = {
    state: (list(CodingInstance.objects.all().filter(project=project.id, policy_instance_id=code.id))+[None])[0]
    for state, code, in codes.items()
}
codings = {state: coding.coding_values for state, coding in codings.items() if coding}

In [22]:
codes["Arizona"].content[0].keys()

dict_keys(['title', 'content', 'ordinal', 'settings'])

In [49]:
def getText(state):
    content = codes[state].content
    to_ret = []
    for doc in content:
        to_ret.append("# " + doc.get("title"))
        for para in doc['content']:
            if para['type'] == 'text':
                to_ret.append("  ".join(para['content']))
            elif para['type'] == 'list':
                for bullet in para['content']:
                    to_ret.append("  ".join(bullet['content']))
            elif para['type'] == 'section':
                to_ret.append("#"*para['level'] + " " + "  ".join(para['content']))
            else:
                print(para)
    return '\n'.join(to_ret)

for state in codes.keys():
    with open(f"/Users/stein/Desktop/states/{state}.txt", "w") as f:
        f.write(getText(state))t